<a href="https://colab.research.google.com/github/PavelStelmakhV/hw306-pyspark/blob/main/Hw6.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=472576486b434cf5fcbe3c351437a8af539a279ac42db438578eacd890a6b63c
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [2]:
from google.colab import drive
from google.colab import files


drive.mount('/content/drive')
path = '/content/drive/My Drive/'
file_name = 'bestsellers with categories.csv'

Mounted at /content/drive


In [9]:
import pandas as pd
import pyspark
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, stddev, round as _round
from pyspark.sql.types import *


spark = SparkSession.builder.getOrCreate()

# spark = SparkSession.builder\
        # .master("local[*]")\
        # .appName('Hw6.1')\
        # .getOrCreate()

In [11]:
data_schema = [
               StructField('name', StringType(), True),
               StructField('author', StringType(), True),
               StructField('user_rating', FloatType(), True),
               StructField('reviews', IntegerType(), True),
               StructField('price', FloatType(), True),
               StructField('year', IntegerType(), True),
               StructField('genre', StringType(), True),
            ]

final_struc = StructType(fields = data_schema)

In [12]:
csv_file = path + file_name
df = spark.read.csv(csv_file, header=True, schema=final_struc)

In [13]:
df.show()

+--------------------+--------------------+-----------+-------+-----+----+-----------+
|                name|              author|user_rating|reviews|price|year|      genre|
+--------------------+--------------------+-----------+-------+-----+----+-----------+
|10-Day Green Smoo...|            JJ Smith|        4.7|  17350|  8.0|2016|Non Fiction|
|   11/22/63: A Novel|        Stephen King|        4.6|   2052| 22.0|2011|    Fiction|
|12 Rules for Life...|  Jordan B. Peterson|        4.7|  18979| 15.0|2018|Non Fiction|
|1984 (Signet Clas...|       George Orwell|        4.7|  21424|  6.0|2017|    Fiction|
|5,000 Awesome Fac...|National Geograph...|        4.8|   7665| 12.0|2019|Non Fiction|
|A Dance with Drag...| George R. R. Martin|        4.4|  12643| 11.0|2011|    Fiction|
|A Game of Thrones...| George R. R. Martin|        4.7|  19735| 30.0|2014|    Fiction|
|A Gentleman in Mo...|         Amor Towles|        4.7|  19699| 15.0|2017|    Fiction|
|A Higher Loyalty:...|         James Comey|

Используя только RDD вычислить среднюю цену всех книг, у которых рейтинг выше 4.7

In [7]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- User Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Genre: string (nullable = true)



In [15]:
df.dtypes

[('name', 'string'),
 ('author', 'string'),
 ('user_rating', 'float'),
 ('reviews', 'int'),
 ('price', 'float'),
 ('year', 'int'),
 ('genre', 'string')]

In [18]:
df.describe().show()

+-------+--------------------+----------------+-------------------+------------------+------------------+------------------+-----------+
|summary|                name|          author|        user_rating|           reviews|             price|              year|      genre|
+-------+--------------------+----------------+-------------------+------------------+------------------+------------------+-----------+
|  count|                 550|             550|                550|               550|               550|               550|        550|
|   mean|                null|            null| 4.6183636452934955|11953.281818181818|              13.1|            2014.0|       null|
| stddev|                null|            null|0.22698038875323445|11731.132017431892|10.842261978422364|3.1651563841692782|       null|
|    min|"The Plant Parado...|Abraham Verghese|                3.3|                37|               0.0|              2009|    Fiction|
|    max|You Are a Badass:...|    Zhi Gan

In [19]:
df.filter(df.user_rating >= 4.7).show()

+--------------------+--------------------+-----------+-------+-----+----+-----------+
|                name|              author|user_rating|reviews|price|year|      genre|
+--------------------+--------------------+-----------+-------+-----+----+-----------+
|5,000 Awesome Fac...|National Geograph...|        4.8|   7665| 12.0|2019|Non Fiction|
|  Alexander Hamilton|         Ron Chernow|        4.8|   9198| 13.0|2016|Non Fiction|
|Barefoot Contessa...|          Ina Garten|        4.8|   1296| 24.0|2012|Non Fiction|
|            Becoming|      Michelle Obama|        4.8|  61133| 11.0|2018|Non Fiction|
|            Becoming|      Michelle Obama|        4.8|  61133| 11.0|2019|Non Fiction|
|Being Mortal: Med...|        Atul Gawande|        4.8|  11113| 15.0|2015|Non Fiction|
|Brown Bear, Brown...|     Bill Martin Jr.|        4.9|  14344|  5.0|2017|    Fiction|
|Brown Bear, Brown...|     Bill Martin Jr.|        4.9|  14344|  5.0|2019|    Fiction|
|Cabin Fever (Diar...|         Jeff Kinney|